In [1]:
import matplotlib 
get_ipython().magic(u'matplotlib inline')

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import pyLDAvis

from __future__ import print_function
from time import time
from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

ImportError: No module named 'pyLDAvis'

In [2]:
n_samples = 2000
n_features = 10000
n_topics = 10
n_top_words = 100

from nltk.corpus import stopwords
mystopwords = stopwords.words("english")

This will set up the document using the .txt files that you have in sharepoint, below is the version for excel.

In [8]:
#MyPath = r"C:\Users\williamh\OneDrive - METIA LTD\LinkedIn\{}" #for running this code on my work PC
MyPath = ""


files = [file for file in os.listdir(MyPath.format("")) if ".txt" in file]

Healthcare = [file for file in files if "Health" in file]
Finance = [file for file in files if "Finance" in file]

hdf = {}
fdf = {}
alldf = {}

for file in Healthcare:
    print(file)
    with open (MyPath.format(file), "r") as myfile:
        data="".join([line.rstrip('\n') for line in myfile])
    i = len(hdf)
    hdf[file] = data
    
for file in Finance:
    print(file)
    with open (MyPath.format(file), "r") as myfile:
        data="".join([line.rstrip('\n') for line in myfile])
    i = len(fdf)
    fdf[file] = data
    
for file in files:
    print(file)
    with open (MyPath.format(file), "r") as myfile:
        data="".join([line.rstrip('\n') for line in myfile])
    i = len(alldf)
    alldf[file] = data


FileNotFoundError: [Errno 2] No such file or directory: ''

In [4]:
#df = pd.read_excel('Copy of LinkedIn_InterviewCategorization_111816.xlsx')
df = pd.read_excel('Copy of LinkedIn_InterviewCategorization_1118113016.xlsx')
df.index = df['Interview']
df = df.drop(['Interview'],axis=1)
df = df.replace(np.nan,' ', regex=True)

In [12]:
df.head()

,Background & Work Environment,Challenges,Roles of Content,Types of Content,Content Consumption Habits,Live Community,Social,LinkedIn
Interview,,,,,,,,
"Nov. 9, 2016 – Interview 5 - Finance",Sure. I knew I was always going to be an accou...,I think getting people to understand what I'm ...,"Oh, it plays a huge role. It changes every day...",Sure. I mean if we're talking about looking up...,"on my phone, I can ask Siri just about anythin...","Oh, I highly believe it's not what you know, i...","But if I'm looking for something specific, I u...","Yeah, I use it to... Well, one, I do monitor e..."
"Nov 11, 2016 - Interview 6 - Finance",With 1.5 billion. I’m an equity portfolio mana...,One of the big drawbacks of Google right now i...,The reasons would be obviously independent of ...,"I guess for information, maybe just a little b...","It’s not going to sound like a lot; but again,...",,but one of the really impressive things with T...,"Yes. The best that I can. For the most part, I..."
"Nov 9, 2016 - Interview 5 - Healthcare",I am Associate Professor of Surgery in the Uni...,"Yes, time restrictions. The schedule is gettin...","You want specifics, what I use, or how importa...","Well, let’s see what type of resources. If we ...","If I have specific devices, you said, also? Ye...",,I will not discuss about Facebook because I do...,So I am like a member of LinkedIn for the past...
"Nov 10, 2016 - Interview 8 - Healthcare","Sure, so I deliver anesthesia. I’m board-certi...",I think the big one is the shift in culture an...,"Sure. Well so, in healthcare there’s seven dif...","Well, the history behind it is about 20 years ...","Oh yeah, all the time. I’m always hooked in. \...","Well, we’re looking at two different versions ...",… I teach my grad students when it comes to or...,I’m just the basic professional profile. I rea...
"Nov 10, 2016 - Interview 9 - Healthcare",…And went on to work in an outpatient setting ...,Some of the challenges that I face in my role…...,"Obtaining content or information, if I have a ...",We use the UpToDate resource manuals. I don’t ...,Sure. Online I just feel it’s at the tip of my...,Say I’m having a really tough day. I know nurs...,I know one of the topics that was in the e-mai...,"No. To be honest with you, no. I remember for ..."


In [13]:
sortcorp = {}

for row in df.index:
    for col in df.columns.tolist():
        idx = row +'::'+ col
        sortcorp[idx] = df.loc[row,col]

## Doing this for one corpus at a time:
change this line to switch between corpus


I added some new stopwords:

In [14]:
corpus = sortcorp


In [15]:
data_samples = corpus.values()
tf_vectorizer = CountVectorizer(strip_accents='unicode',
                                lowercase= True,
                                token_pattern=r'\b[a-zA-Z]{3,}\b',
                                max_features=n_features,
                                stop_words=mystopwords,
                                max_df=0.5,
                                min_df=10)

tf = tf_vectorizer.fit_transform(data_samples)

In [16]:
print("Fitting LDA models with tf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                                learning_method='online', learning_offset=50.,
                                random_state=0)
lda.fit(tf)
tf_feature_names = tf_vectorizer.get_feature_names()

Fitting LDA models with tf features, n_samples=2000 and n_features=10000...


In [23]:
#bunch of functions that I wrote to build the tables for the model:
def get_single_topic(lda, tf_feature_names, n_top_words, topic):
    words = [tf_feature_names[i] for i in lda.components_[topic].argsort()[:-n_top_words - 1:-1]]
    scores = lda.components_[topic][lda.components_[topic].argsort()[:-n_top_words - 1:-1]]
    df = pd.DataFrame(index=words,columns=['topic_{}'.format(topic)],data=scores)
    return df

def get_all_topics(lda, tf_feature_names, n_top_words,n_topics):
    df = pd.DataFrame()
    for topic in range(n_topics):
        tmpdf = get_single_topic(lda, tf_feature_names, n_top_words, topic)
        for item in tmpdf.index:
            df.loc[item,'topic_{}'.format(topic)] = tmpdf.loc[item,'topic_{}'.format(topic)]
    return df

def get_topic_names(lda, tf_feature_names, n_top_words, n_topics):
    themes = pd.Series()
    for topic in range(n_topics):
        theme = " ".join([tf_feature_names[i] for i in lda.components_[topic].argsort()[:-n_top_words - 1:-1]])
        themes.loc['topic_{}'.format(topic)] = theme
    return themes

words = get_all_topics(lda, tf_feature_names, n_top_words,n_topics)

In [18]:
def score_document(doc_dic,df,lda, tf_feature_names, n_top_words, n_topics,
                    returnDF=True,confidence=.01):
    '''
    gives scores to the origional document, assigning a category to each one. 

    returnDF : By default returns a DataFrame, set to false to return a dict.
    confidence : this is the threashold that the model must meet to match the document to a topic.
    set to .01 to include practically everything, set to .99 to include almost nothing.

    document_scores = score_document(doc_dic,df,lda, tf_feature_names, n_top_words, n_topics)
    '''
    results_dict = {}
    for key, item in enumerate(doc_dic.keys()):
        document = doc_dic[item]
        words = [tf_feature_names[i] for i in tf.getrow(list(doc_dic).index(item)).indices]
        scores = df[[word in words for word in df.index]]
        TM_Score = pd.DataFrame()
        TM_Score['docScore'] = scores.sum() 
        TM_Score['globalScore'] = df.sum()
        TM_Score['relevance'] = TM_Score['docScore']/TM_Score['globalScore']
        TM_Score['theme'] = get_topic_names(lda, tf_feature_names, n_top_words, n_topics)
        results = TM_Score['relevance'].fillna(0).to_dict()
        results['document'] = document
        results['key'] = item
        results['top_score'] = TM_Score['relevance'].max()

        if TM_Score['relevance'].max() >= confidence:
            results['top_theme'] = TM_Score['theme'][TM_Score['relevance'].tolist().index(TM_Score['relevance'].max())]
            results['top_topic'] = TM_Score.index[TM_Score['relevance'].tolist().index(TM_Score['relevance'].max())]
        else: 
            results['top_theme'] = 'unassigned'
            results['top_topic'] = 'unassigned'
        results_dict[key] = results
    if returnDF:
        return pd.DataFrame(results_dict).T
    else:
        return results_dict

    
Scored_corpus = score_document(corpus,words,lda, tf_feature_names, n_top_words, n_topics)

In [19]:
Scored_corpus['interveiw'] = Scored_corpus['key'].apply(lambda x: x.split("::")[0])
Scored_corpus['conversation'] = Scored_corpus['key'].apply(lambda x: x.split("::")[1])
Scored_corpus.head()

,document,key,top_score,top_theme,top_topic,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,interveiw,conversation
0,"They are rather a pretty important part, like...","November 7th, 2016 - Interview 1 - Finance Man...",0.792395,like would think kind information know really ...,topic_9,0.523273,0.653863,0.65777,0.157526,0.630594,0.682426,0.600344,0.749704,0.775597,0.792395,"November 7th, 2016 - Interview 1 - Finance Man...",Roles of Content
1,Well my current role is that I've been an owne...,"November 8th, 2016 - Interview 2 - Financial P...",0.278393,like would think get kind know different infor...,topic_8,0.259493,0.219581,0.224594,0.14637,0.252046,0.218524,0.176497,0.232671,0.278393,0.220622,"November 8th, 2016 - Interview 2 - Financial P...",Background & Work Environment
2,"Oh yeah, all the time. I’m always hooked in. \...","Nov 10, 2016 - Interview 8 - Healthcare::Conte...",0.827682,like would think kind information know really ...,topic_9,0.763549,0.610349,0.644218,0.109392,0.477082,0.746416,0.622737,0.807018,0.814217,0.827682,"Nov 10, 2016 - Interview 8 - Healthcare",Content Consumption Habits
3,… I teach my grad students when it comes to or...,"Nov 10, 2016 - Interview 8 - Healthcare::Social",0.690475,know people think something would use informat...,topic_5,0.636644,0.524978,0.526024,0.302186,0.508312,0.690475,0.574269,0.575996,0.595668,0.600739,"Nov 10, 2016 - Interview 8 - Healthcare",Social
4,"A lot of the time also, if I feel like I use r...","Nov. 8, 2016 – Interview 7 - Healthcare::Live ...",0.36862,like would think get kind know different infor...,topic_8,0.207338,0.261229,0.281715,0.0481338,0.255973,0.262299,0.247508,0.326976,0.36862,0.340126,"Nov. 8, 2016 – Interview 7 - Healthcare",Live Community


In [20]:
get_topic_names(lda, tf_feature_names, n_top_words, n_topics)

topic_0    information would lot kind one get going diffe...
topic_1    get well one website information think know lo...
topic_2    like linkedin would know facebook think kind r...
topic_3    think material gaining highly 18 accomplishmen...
topic_4    kind like know would things really search work...
topic_5    know people think something would use informat...
topic_6    kind really linkedin think like know lot use o...
topic_7    like people go think things would something ge...
topic_8    like would think get kind know different infor...
topic_9    like would think kind information know really ...
dtype: object

Output to .csv files

In [25]:
words.to_csv(r'token_list.csv')

Scored_corpus.to_csv(r'Scored_corpus.csv')